In [ ]:
# create workspace
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset, Datastore, ScriptRunConfig
from azureml.data.data_reference import DataReference
import os

In [ ]:
# create a pipeline
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.pipeline.core import PipelineParameter
from azureml.core.datastore import Datastore
from azureml.pipeline.core import PipelineData
from azureml.data import OutputFileDatasetConfig
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.core import Environment


ws = Workspace.from_config()
datastore = ws.get_default_datastore()

#Retrieve an already attached Azure Machine Learning Compute
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except ComputeTargetException:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 0, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# get datasets from CI
# Ad hoc Datasets also referred to Saved datasets are not discoverable from Audience Insights currently.
# Example: IncompatibleDataset = Dataset.Tabular.from_delimited_files('https://cidatasets.blob.core.windows.net/hotel-scenario/ContosoHotel_HotelActivity_1.csv')
# You need to use registered datasets and then consume them by their name 
# Example: CompatibleDataset = Dataset.get_by_name(ws, name='Hotel Stay Activity Data')

customer_dataset = Dataset.get_by_name(ws, name='CustomerData')
customer_pipeline_param = PipelineParameter(name="Customer_pipeline_param", default_value=customer_dataset)
customer_ds_consumption = DatasetConsumptionConfig("customer_dataset", customer_pipeline_param)


OutputPathParameter = PipelineParameter(name="output_path", default_value="LeaseRenewalPredictionOutput4/LeaseRenewalPredictionOutput4.csv")
OutputDatastoreParameter = PipelineParameter(name="output_datastore", default_value="workspaceblobstore")

env = Environment.from_conda_specification(name = 'env', file_path = './InferencePipeline.yml')

# source_directory
source_directory = './scripts'
os.makedirs(source_directory, exist_ok=True)
script_name = "score.py"
# define a single step pipeline for demonstration purpose.
src = ScriptRunConfig(source_directory=source_directory,
                    script=script_name,
                    compute_target=aml_compute_target,
                    environment=env)

inferenceStep = PythonScriptStep(
    name="Inferencing_Step",
    script_name=src.script,
    #arguments=["--input_data1", customer_ds_consumption,"--input_data2", resident1_ds_consumption,"--input_data3", resident2_ds_consumption, "--input_data4", lease_ds_consumption, "--input_data5",payment_ds_consumption,"--input_data6", survey_ds_consumption,"--input_data7", workorder_ds_consumption, "--output_path", OutputPathParameter, "--output_datastore", OutputDatastoreParameter],
    arguments=["--input_data1", customer_ds_consumption, "--output_path", OutputPathParameter, "--output_datastore", OutputDatastoreParameter],
    #inputs=[customer_ds_consumption, resident1_ds_consumption, resident2_ds_consumption, lease_ds_consumption, payment_ds_consumption,survey_ds_consumption,workorder_ds_consumption], 
    inputs=[customer_ds_consumption], 
    #compute_target=aml_compute_target, 
    source_directory=src.source_directory,
    runconfig = src.run_config)

print("Inferencing_Step created")
# build and validate Pipeline
pipeline = Pipeline(workspace=ws, steps=[inferenceStep])
print("Pipeline is built")

In [ ]:
from azureml.pipeline.core import PipelineEndpoint
endpoint_name = 'LeaseRenewalPipelineEndpoint'
endpoints_available = PipelineEndpoint.list(ws, active_only=False)
for i in endpoints_available:
    if i.name == endpoint_name:
        print(i.name)
        i.disable()
        i.archive()

In [ ]:
# publish pipeline endpoint
from azureml.pipeline.core import PipelineEndpoint
pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="LeaseRenewalPipelineEndpoint", pipeline=pipeline, description="Lease Renewal Pipeline Endpoint")
#published_pipeline = pipeline.publish(name="LeaseRenewalPipeline", description="published pipeline")

In [ ]:
# pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="LeaseRenewalPipeline") 
# pipeline_endpoint.add_default(pipeline=published_pipeline)

In [ ]:
from azureml.core import Experiment
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="LeaseRenewalPipelineEndpoint")
experiment = Experiment(ws, 'Lease-Renewal-Exp_Inference')
pipeline_run = experiment.submit(config=pipeline_endpoint)
pipeline_run.get_details()